In [13]:
import time
import csv
import random
!pip install selenium==4.14.0
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
!pip install webdriver-manager==4.0.0
from webdriver_manager.chrome import ChromeDriverManager
!pip install langdetect==1.0.9
from langdetect import detect
!apt-get update
!apt-get install -y google-chrome-stable
import os
os.environ['PATH'] += ":/usr/bin/google-chrome"
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#chrome_options.binary_location = "/usr/bin/google-chrome"

# Setup Chrome in headless mode
#chrome_options = Options()
#chrome_options.binary_location = "/usr/bin/google-chrome"  # Path to Chrome executable in Colab
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize WebDriver
driver = webdriver.Chrome(options=chrome_options)

# List of TV show URLs to scrape
tv_show_urls = [
    "hhttps://www.metacritic.com/tv/adolescence/user-reviews/",
    "https://www.metacritic.com/tv/reacher/user-reviews/",
    "https://www.metacritic.com/tv/stranger-things/user-reviews",
    "https://www.metacritic.com/tv/the-office/user-reviews",
    "https://www.metacritic.com/tv/friends/user-reviews"
]

def scrape_reviews(url, max_pages=5):
    """
    Scrape user reviews from a given Metacritic TV show URL.

    Args:
        url (str): The URL of the TV show's user reviews page.
        max_pages (int): Maximum number of pages to scrape.

    Returns:
        tuple: Lists of positive, mixed, and negative reviews.
    """
    positive_reviews = []
    mixed_reviews = []
    negative_reviews = []
    page = 0

    while page < max_pages:
        driver.get(f"{url}?page={page}")

        try:
            # Wait until reviews are loaded
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-testid='product-review']"))
            )
        except:
            print(f"No reviews found on page {page}. Stopping.")
            break

        # Scroll down to load dynamic content
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Allow content to load
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract review elements
        review_containers = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='product-review']")

        for review in review_containers:
            try:
                # Extract review text
                review_text_element = review.find_element(By.CSS_SELECTOR, "div.c-siteReview_quote span")
                review_text = review_text_element.text.strip()

                # Extract rating
                rating_element = review.find_element(By.CSS_SELECTOR, "div.c-siteReviewHeader_reviewScore")
                rating = rating_element.text.strip()

                # Ensure review is in English
                if review_text and detect(review_text) == "en":
                    rating = int(rating)
                    if rating >= 7:
                        positive_reviews.append((rating, review_text, "Positive"))
                    elif 4 <= rating <= 6:
                        mixed_reviews.append((rating, review_text, "Mixed"))
                    else:
                        negative_reviews.append((rating, review_text, "Negative"))
            except Exception as e:
                continue  # Skip errors and continue scraping

        page += 1

    return positive_reviews, mixed_reviews, negative_reviews

# **Scraping reviews from multiple TV shows**
final_positive_reviews = []
final_mixed_reviews = []
final_negative_reviews = []
random.shuffle(tv_show_urls)  # Shuffle URLs for diversity

for tv_show_url in tv_show_urls:
    print(f"Scraping: {tv_show_url}")
    pos, mix, neg = scrape_reviews(tv_show_url)
    final_positive_reviews.extend(pos)
    final_mixed_reviews.extend(mix)
    final_negative_reviews.extend(neg)
    print(f"Total reviews collected so far: {len(final_positive_reviews) + len(final_mixed_reviews) + len(final_negative_reviews)}")
    if len(final_positive_reviews) + len(final_mixed_reviews) + len(final_negative_reviews) >= 15000:
        break

# **Display a sample of extracted reviews**
print("\n--- Sample Extracted Reviews ---\n")
all_reviews_sample = final_positive_reviews[:3] + final_mixed_reviews[:3] + final_negative_reviews[:3]
for idx, (rating, review, sentiment) in enumerate(all_reviews_sample):
    print(f"Review {idx + 1}: [Rating: {rating}] [{sentiment}] {review}")
    print("-" * 100)

# **Save extracted reviews to CSV**
csv_filename = "tv_reviews.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Rating", "Review", "Sentiment"])
    writer.writerows(final_positive_reviews + final_mixed_reviews + final_negative_reviews)

print(f"\n Successfully extracted and saved reviews to {csv_filename}!")

# **Close WebDriver**
driver.quit()


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,540 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,081 kB]
Fetched 4,878 kB in 1s (3,581 kB/s)
Reading package lists... Done
W: Skipping acquire of configur

In [14]:
from google.colab import files
files.download("tv_reviews.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>